In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1579234887099_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#import pyspark

#from pysparkling import *

ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
data_path = "s3://msds694-group15/slow.csv"
data = ss.read.csv(data_path, inferSchema=True, header=True)

processed_rdd = data.rdd\
                 .map(lambda x: list(x[0:11])
                      + [min(x[11:]), max(x[11:]), float(np.mean(x[11:])), float(np.median(x[11:]))]
                      + [x[-1]])

schema = StructType([StructField("num_prev_so", IntegerType(), True),
                     StructField("mean_amp_prev_so", DoubleType(), True),
                     StructField("mean_duration_prev_so", DoubleType(), True),
                     StructField("amp_curr_so", DoubleType(), True),
                     StructField("duration_curr_so", IntegerType(), True),
                     StructField("curr_sleep_stage", IntegerType(), True),
                     StructField("time_elapsed_since_asleep", IntegerType(), True),
                     StructField("time_deep_sleep", IntegerType(), True),
                     StructField("time_light_sleep", IntegerType(), True),
                     StructField("time_rem_sleep", IntegerType(), True),
                     StructField("time_wake_sleep", IntegerType(), True),
                     StructField("eeg_min", DoubleType(), True),
                     StructField("eeg_max", DoubleType(), True),
                     StructField("eeg_mean", DoubleType(), True),
                     StructField("eeg_median", DoubleType(), True),
                     StructField("label", IntegerType(), True)
                    ])

data_preprocessed = ss.createDataFrame(processed_rdd, schema)
data_preprocessed.printSchema()

data_preprocessed.show(5)

data_preprocessed.write.option("path", "/data").saveAsTable('data17')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- num_prev_so: integer (nullable = true)
 |-- mean_amp_prev_so: double (nullable = true)
 |-- mean_duration_prev_so: double (nullable = true)
 |-- amp_curr_so: double (nullable = true)
 |-- duration_curr_so: integer (nullable = true)
 |-- curr_sleep_stage: integer (nullable = true)
 |-- time_elapsed_since_asleep: integer (nullable = true)
 |-- time_deep_sleep: integer (nullable = true)
 |-- time_light_sleep: integer (nullable = true)
 |-- time_rem_sleep: integer (nullable = true)
 |-- time_wake_sleep: integer (nullable = true)
 |-- eeg_min: double (nullable = true)
 |-- eeg_max: double (nullable = true)
 |-- eeg_mean: double (nullable = true)
 |-- eeg_median: double (nullable = true)
 |-- label: integer (nullable = true)

+-----------+------------------+---------------------+-----------+----------------+----------------+-------------------------+---------------+----------------+--------------+---------------+--------+-------+--------------------+-----------------+-----+
|num_pr

In [5]:
import time
start_time = time.time()

data = ss.sql("select * from parquet.`/data`")

#sc.list_packages()

sc.install_pypi_package("h2o-pysparkling-2.4")

sc.install_pypi_package("h2o")

from pysparkling import *
import pyspark

import h2o
h2o.init()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for h2o-pysparkling-2.4: filename=h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl size=101307807 sha256=8c4a787f9bfe999f19c1d0f76d8b087d7cf6c44d7e482016dd0be7cdc31f97c3
  Stored in directory: /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765
  Created wheel for tabulate: filename=tabulate-0.8.6-cp36-none-any.whl size=23271 sha256=a5bbbbfa6767a0db7e0bbe8e1d0a422015b57c6debcbbaef32ff3a7c9a89a04d
  Stored in directory: /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b444d976d7ce5a2c9496010cbc82c8aab5a29292a34606b524efd5d315ba0742
  Stored in directory: /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=602e8080a7acd60c17b9d62ae66f4cf64053534ffa0b07bc8a69a4adc10

In [6]:
ss = pyspark.sql.SparkSession.builder.master('yarn')\
.config("spark.dynamicAllocation.enabled", True)\
.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
hc = H2OContext.getOrCreate(ss)

h2o_df = hc.as_h2o_frame(data, "sleepData")

### split dataset

train_df, valid_df = data.randomSplit([0.85, 0.15], seed=1)
train_df.cache()
valid_df.cache()

train_h2o = hc.as_h2o_frame(train_df, 'train_h2o')
valid_h2o = hc.as_h2o_frame(valid_df, 'valid_h2o')

train_h2o['label'] = train_h2o['label'].asfactor()
valid_h2o['label'] = valid_h2o['label'].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Connecting to H2O server at http://ip-172-31-11-230.us-west-1.compute.internal:54323 ... successful.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         04 secs
H2O cluster timezone:       Etc/UCT
H2O data parsing timezone:  UTC
H2O cluster version:        3.28.0.1
H2O cluster version age:    1 month
H2O cluster name:           sparkling-water-livy_application_1579234887099_0001
H2O cluster total nodes:    1
H2O cluster free memory:    18.46 Gb
H2O cluster total cores:    8
H2O cluster allowed cores:  4
H2O cluster status:         accepting new members, healthy
H2O connection url:         http://ip-172-31-11-230.us-west-1.compute.internal:54323
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, Amazon S3, AutoML, Core V3, TargetEncoder, Core V4
Python version:             3.6.8 final
--------------------------  ---------------------------------------------------------

In [8]:
predictors = train_h2o.names[:]
response = train_h2o.names[-1]
predictors.remove(response)

n_folds = 3
fold_assignment = "Modulo"
keep_cross_validation_predictions = True

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
start_time_model = time.time()
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

model_dl = H2ODeepLearningEstimator(variable_importances=True,
                                    nfolds=n_folds,
                                    fold_assignment=fold_assignment,
                                    keep_cross_validation_predictions=keep_cross_validation_predictions,
                                    loss="Automatic")

model_dl.train(x=predictors,
               y=response,
               training_frame=train_h2o)

end_time_total = time.time() - start_time
end_time_model = time.time() - start_time_model
print (end_time_total)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'end_time' is not defined
Traceback (most recent call last):
NameError: name 'end_time' is not defined



In [10]:
start_time_GB = time.time()
from h2o.estimators.gbm import H2OGradientBoostingEstimator

model_gbm = H2OGradientBoostingEstimator(ntrees=50,
                                         max_depth=6,
                                         learn_rate=0.1,
                                         nfolds=n_folds,
                                         fold_assignment=fold_assignment,
                                         keep_cross_validation_predictions=keep_cross_validation_predictions,
                                         distribution="multinomial")
model_gbm.train(x=predictors,
                y=response,
                training_frame=train_h2o)

end_time_GB = time.time() - start_time_GB

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gbm Model Build progress: [###############################################] 100%

In [11]:
print (end_time_total)
print (end_time_model)
print (end_time_GB)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

292.74765825271606
203.1637613773346
58.72330665588379

In [12]:
model_dl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1579235981139_1


Status of Neuron Layers: predicting label, 3-class classification, multinomial distribution, CrossEntropy loss, 44,003 weights/biases, 525.5 KB, 2,301,704 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms             momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  -------------------  ----------  ---------------------  -------------------  --------------------  -------------------
    1        15       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.013772237491813333  0.01157400757074356  0.0         -0.004195501443454608  0.1740310788154602   -0.568648045584691    0.22358930110931396
    3        200      Rectifier  0.0        0.0   0.0   0.09963003917039459   0.

In [14]:
model_gbm

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1579235981139_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 150                         119742                 6            6            6             31            64            58.9067


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.3711089997005039
RMSE: 0.6091871631120471
LogLoss: 0.968923223965099
Mean Per-Class Error: 0.5221222967577579

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0       1      2      Error     Rate
------  -----  -----  --------  -----------------
77340   13133  6422   0.201816  19,555 / 96,895
32364   22856  